In [ ]:


// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

import better.files._
import better.files.Dsl._ 

// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
assert(File(jarpath).exists, "must run jupyter notebook from docker")
interp.load.cp(os.Path(jarpath))

In [ ]:
import almond.interpreter.api.DisplayData
import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._


import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}
import edu.colorado.plv.bounder.lifestate.LifeState
import edu.colorado.plv.bounder.lifestate.LifeState.{Not,And}
import edu.colorado.plv.bounder.ir.{CIEnter,CIExit,CBEnter,CBExit}

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil,Action} // Historia utilities
import upickle.default.read
import upickle.default.write
import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,MemoryLeak,NamedPureVar,TopVal, InitialQueryWithStackTrace, Reachable}
import edu.colorado.plv.bounder.lifestate.SAsyncTask
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, 
                                           SDialog, SpecSignatures, SpecSpace, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature,SubClassMatcher}
import edu.colorado.plv.bounder.symbolicexecutor.{PreciseApproxMode,LimitAppRecursionDropStatePolicy,LimitMsgCountDropStatePolicy,
                                                  DumpTraceAtLocationPolicy,LimitLocationVisitDropStatePolicy, LimitCallStringDropStatePolicy, 
                                                  LimitMaterializedFieldsDropStatePolicy}
import edu.colorado.plv.bounder.symbolicexecutor.Z3TimeoutBehavior

def getIsDocker():Boolean = {
    val res = BounderUtil.runCmdStdout("whoami")
    res.trim == "root"
}

val isDocker = getIsDocker()
// val isDocker = false //TODO: overridden

val historiaDir = if(isDocker) "/home/bounder" else "/Users/shawnmeier/Documents/source/historia/Historia/"

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File, printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val javaMemLimit=20 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o ${outputMode} --debug"
    
    if(printThenDone){
        println(cmd)
        cmd
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        BounderUtil.runCmdFileOut(cmd, configPath.parent).toString
    }
}

// def printOutput(

def runAndPrint(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false, outputMode:String = "MEM"):String = {
    val res = runHistoriaWithSpec(configPath,printThenDone, outputMode)
    println("run result")
    println(res)   
    println("specified messages")
    val msgSigs = allSpecs.flatMap(spec =>
                Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)
    println(msgSigs)
    println(msgSigs.size)
    res
}

val v = NamedPureVar("v")
val a = NamedPureVar("a")
val f = NamedPureVar("f")
val s = NamedPureVar("s")
val l = NamedPureVar("l")
val t = NamedPureVar("t")
val r = NamedPureVar("r")
val dataDir = if(isDocker) "/home/testApps/" else "/Users/shawnmeier/Documents/data/reach_24_data/"
val notebooksDir = if(isDocker) "/home/notebooks/" else s"${historiaDir}/notebooks/"


//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}
case class Table(v:Seq[TableAble]){

}
Displayers.register(classOf[Table], (t: Table) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(t.v.head.headers.map(v => th(v))),
        for (row <- t.v) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
              case v:Float => td(v)
          }
        )
      ).render
    }
  ).asJava
})

case class CategoryRow(categoryTitle:String, shortTitle:String, subMeasures: List[(String,Any)]) extends TableAble{
    override def headers:List[String] = subMeasures.map(_._1)
    override def values:List[Any] = subMeasures.map(_._2)
}

case class RuntimeRow(dir:File, catRows:List[CategoryRow]) extends TableAble{
    def benchmarkName:String = dir.toString.split("/").last
    override def headers:List[String] = "benchmark name" :: catRows.flatMap{row => row.headers.map{hdr => s"${row.shortTitle}:${hdr}"}}
    override def values:List[Any] = benchmarkName :: catRows.flatMap{row => row.values}
}

def avg(values:Seq[Int]):Float = {
    val sum = values.sum.toFloat
    println(s"sum ${sum}")
    sum / values.size.toFloat
}
// read and print statistics about runtime traces
// should be a directory with logcat output one trace per file named "logcat.txt", "logcat1.txt" etc
// add "WITNESS" to the end of each logcat file where the issue was reached.
def runtimeStats(dir:File):RuntimeRow = {
    val traces = dir.glob("logcat*txt")
    val traceContents = traces.map{t => (t,t.contentAsString().split("\n"))}.toList
    val traceCount = traceContents.size
    println(s"found ${traceCount} runtime traces")
    val cbRowMatcher = ".*histInstrumentation.*cb.*".r
    val callbackCountPerTrace = avg(traceContents.map{case (tfile, rows) => rows.filter(cbRowMatcher.matches).size})

    val witnessCount = traceContents.filter{case (tfile, rows) => rows.exists{r => r.contains("WITNESS")}}.size
    RuntimeRow(dir, List(CategoryRow("Runtime Inst", "ri", List(
        ("Witness Ratio", s"${witnessCount}/${traceCount}"),
        ("cb(n)", callbackCountPerTrace)
    ))))
}

Table(Seq(
    RuntimeRow(File("/foo/bar"), List(CategoryRow("c1","c1",List(("v1",1),("v2",2))), CategoryRow("c2","c2",List(("v3",3))))),
    RuntimeRow(File("/foo/baz"), List(CategoryRow("c1","c1",List(("v1",4),("v2",5))), CategoryRow("c2","c2",List(("v3",6))))),
))

Memory Leak
-----------

In [ ]:
// Memory leak runtime stats

val memLeakExpBase = s"${dataDir}/ChatGPT_Benchmarks/EXN_activity_leak_noStatic"
val memLeakRuntimeStats = runtimeStats(File(memLeakExpBase))
Table(Seq(memLeakRuntimeStats))

In [ ]:
// Memory Leak


val v = NamedPureVar("v")
val s = NamedPureVar("s")
val m = NamedPureVar("m")
val p = NamedPureVar("p")

def runAPStartService(){

      val query = MemoryLeak("android.app.Activity",
        Signature("com.example.activityleak.LeakActivity$1",
        "void run()"), 39, SpecSignatures.Activity_onDestroy_exit,
        NamedPureVar("a"))



    val inputApk = s"${memLeakExpBase}/app_bug/app/build/intermediates/apk/debug/app-debug.apk"
    val outputDir = File(s"/home/notebooks/reachExpGPT/MemLeak")
    val configOutputDir = File(s"${notebooksDir}/reachExpGPT/MemLeak")

    
    val cfg =  RunConfig(apkPath = inputApk.toString, 
                         timeLimit=Int.MaxValue,
                    outFolder = Some(configOutputDir.toString),
                    initialQuery = List(query), truncateOut=false,
                    specSet = PickleSpec(Set(LifecycleSpec.Activity_onDestroy_last), 
                                             Set(), 
                                             Set(SJavaThreading.runnableI)
                                        ),
                    componentFilter =  None,
                    approxMode = PreciseApproxMode(true, List(LimitLocationVisitDropStatePolicy(3), 
                                                              LimitAppRecursionDropStatePolicy(3),
                                                              LimitMaterializedFieldsDropStatePolicy(Map("*" -> 2)))),
                    z3TimeoutBehavior = Some(Z3TimeoutBehavior().copy(
                            subsumeTryTimeLimit=List(200000), z3InstanceLimit=16))
                        )


    if(!outputDir.isDirectory){
        mkdir(outputDir)
    }
    val cfgPath = (outputDir / "cfg.json")
    val configCfgPath = configOutputDir / "cfg.json"
    cfgPath.overwrite(write(cfg))
    
    val allSpecs = Set(SDialog.disallowDismiss)

    val justPrintCommand = false
    println(runAndPrint(cfgPath, allSpecs,justPrintCommand).replace(cfgPath.toString,configCfgPath.toString))

    if(!justPrintCommand){
        val result = read[Driver.LocResult]((outputDir / "result_0.txt").contentAsString)
        println(result.witnessExplanation)
        println(result.resultSummary)
    }
}
runAPStartService()